# Memoir Close Reading

## Journal de Goncourt

words/phrases to investigate:

(need French translations) the left, the _ left, suppression, mass rising, mobocracy, insurrection, insurgents, National Guards, Republicanism, reg flag, crowd, mob, gang, violence, friendliness, Prussians, Bellevillans, murders, cannon, cannonade, noise, aggressive, drunk, drunkenness, intoxicated, Federals, barricade, altercation, middle-class, upper-class, shout, dispute, fight, corpse, death, wounded, burning, burn, blood, bullets, shells

places:

College de France, Montmartre, Courberoie, Hotel de Ville, Issy, Neuilly, Arc de l'Etoile, Auteuil, Pepiniere barracks, Opera House, Jardin des Plantes, Faubourg Saint-Germain, Monceau, Batignolles, Tuileries, Passy, Palais Royal, Tuileries, Palais de Justice, Preferecture of Police

people:

Jules Valles, Assi, Clement Thomas, Lecomte, Archbishop, Mont-Valerian, MacMahon

timeline of events and selected quotes:

70.09.25: notice posted outside the college de france announcing the formation of the Commune. demands the suppression of the Prefecture of POlice

70.09.30: "liberty, equality, fraternity" appears

70.10.28: National Guards recruiting on a platform with stairs "draped in the red cloth used by wine merchants"

70.10.31: National Guards shouting "Vive la Commune". "The Government is upset, the Commune established, and the list of the man who has been dictating is to be confirmed by universal suffrage." "Vociferous and gesticulating mobocracy" The date "of a revolution." Except - he doesn't mention the Commune again for a while.

71.03.18: "fighting going on at Montmartre" "triumphing insurrection takes possession of Paris." "Hotel de Ville... orators are speaking of putting traitors to death" "Clement Thomas and Lecomte had been shot"

71.03.19: "Army is in full retreat upon Versailles"

71.03.20: Complains about no "National Guard of the Middle Classes"

71.03.31: Jules Valles, Assi (I forget context)

71.04.02: "Cannonade in the direction of Courberoie. Civil War has commenced!" "... hypocritical murders." "... the Bellevillans have been defeated."

71.04.03: More cannonade, mentions becoming accustomed to cannonade.

71.04.04: "Drunkenness of the National Guards is becoming aggressive to passers-by"

71.04.05: "government of the Committee seems to be at an end; yet the cannonade lasted all day around the fort at Issy, where one sees the largest red flag waving"

71.04.07: "it's quite likely the archbishop will be shot tonight". referencing plan to confiscate weapons: "petty preparations for a reign of terror" 

71.04.09: weapons confiscated from citizens

71.04.14: incoming draft mentioned. 

71.04.15: "I am more frightened than I ever was in the time of the Prussians"

71.04.21: Draft

Time skip - journal says the entries are almost exclusively about bombing and hatred of the Communards

71.05.22: barricade "guarded by a few men who look the reverse of energetic". Versailles at Pepiniere barracks. "National Guards and middle-class men are having altercations all over the place." "there is not a single member of the Commune along the whole line of boulevards" "shouts, disputes, and fights" "everybody whom the National Guard finds in the streets is forcibly enlisted", "hear all night the voices of intoxicated National Guards, shoating hoarsely at every passer-by. It was a day when I at last fell into a sleep broken by explosions and nightmares."

71.05.23: Versailles occupy Faubourg Saint-Germain, Monceau, and Batignolles "immense battlefield... engagements which were taking place from Jardin des Plantes as far as Montmartre" "One sees a corpse with a bleeding head, which four men carry by the arms and legs like a packet of dirty linen, taking it from door to door, none of which will open"

71.05.28: He surveys the damage done to Paris. Sees 26 prisoners walking to their execution. Executioners exit "like drunken men"

71.05.29: "one once more feels the movement of Paris coming to life again"

71.05.30: "from time to time we hear frightful noises - houses collapsing, or volleys"




        







In [2]:
import re
import pickle
from helpers import *

# Elie - 1st example
f = open('memoirs_complete\\elie.txt', 'r')
memoir = f.read()

# find potential contractions
# find_potential_contractions(memoir)

# lowercase
memoir = memoir.lower()

# Cut out introductory stuff - will differ based on memoir
memoir = memoir[memoir.find("journal de la commune") + len("journal de la commune"):]

# Cut out ending stuff - will differ based on memoir
memoir = memoir[:memoir.rfind("et le martyre commence.") + len("et le martyre commence.")]

# cut out stuff between pages - will differ based on memoirs
memoir = re.sub(r'\sjournal\sde\sla\scommune\s[0-9]+', '', memoir)
memoir = re.sub(r'[0-9]+\sjournal\sde\sla\scommune\s', '', memoir)

# create paragraph breaks - involves reducing all 2+ consecutive linebreaks into just 2, treating paragraph break as \n\n 
memoir = re.sub('(\n\s\n)','\n\n',memoir)
consecutive_linebreak_checker = re.compile('(\n)\2{2,}')
memoir = re.sub(consecutive_linebreak_checker,'\n\n',memoir)
memoir = re.sub('(-\n[a-z])','',memoir)
memoir = re.sub('\n\n',' <PB> ',memoir)
memoir = re.sub('\n',' ',memoir)

# fix common mispellings
memoir = memoir.replace(" i[ "," il ")
memoir = memoir.replace(" [l "," il ")
memoir = memoir.replace(" ]l "," il ")
memoir = memoir.replace(" i] "," il ")
memoir = memoir.replace(" 1] "," il ")
memoir = memoir.replace(" ÿ ", " y ")
memoir = memoir.replace(" | es ", " les")

# spell out common abbreviations (trying to avoid punctuation due to sentence separator)
memoir = memoir.replace(" m. "," monsieur ")
memoir = memoir.replace(" mm. ", "messieurs")
memoir = memoir.replace(" mme. ", "madame")

# create sentence breaks - involves treating all instances of <letter><punctuation> <letter> as sentence break (variations allowed, eg ' end sentence." start sentence')
memoir = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", memoir)
memoir = [sentence.strip().replace(" <PB> ", " ") for sentence in memoir] # remove paragraph breaks inside of sentences (because they shouldn't be there; likely due to pagebreak)
memoir = ' <SB> '.join(memoir)
while '  ' in memoir:
    memoir = memoir.replace('  ', ' ')
memoir = memoir.replace(" <SB> <PB> ", " <PB> ")

# list of words and phrases we're interested in
wois = ["commune"]
# list of words and phrase we certainly don't want spellchecker to 'fix'
preserved = ["<PB>", "<SB>"]

# split text into tokens, prepare for ocr error correction and lemmatization
memoir_tokens = memoir.split()

# save half-preprocessed memoir
with open('memoirs_pre_spellcheck/elie.pkl', 'wb') as f:
    pickle.dump(memoir_tokens, f)

# create corpus-wide dictionary of misspelled wois -> properly spelled wois
##memoir_tokens_unique = set(memoir_tokens)
##find_misspelled_wois(all_memoir_tokens_unique, wois, preserved)

# bring in spellcheck dictionary
with open('spell_fixer.pkl', 'rb') as f:
    spell_fixer = pickle.load(f)
# continue pre-processing corpus
with open('memoirs_pre_spellcheck/elie.pkl', 'rb') as f:
    memoir_tokens = pickle.load(f)
memoir_tokens = [spell_fixer[token] if token in spell_fixer.keys() else token for token in memoir_tokens]
memoir_tokens

# convert memoir into string for later processing
memoir_processed = ' '.join(memoir_tokens)
with open('memoirs_preprocessed/elie.pkl', 'wb') as f:
    pickle.dump(memoir_processed, f)


In [3]:
memoir_processed





'<PB> paris, 20 mars 1871. <PB> préparé de longue main par monsieur thiers et ses complices, le coup d\'etat qu\'on sentait confusément avancer dans l\'ombre a enfin éclaté et avorté. <SB> on a trouvé dans les ministères des télégrammes échangés entre thiers et favre, entre le ministre de la guerre et le préfet de police, entre l\'honorable jules favre et l\'honoable jules ferry. <SB> nous en donnons quelques extraits. <SB> d\'abord lu série thiers à jules favre qui, malheureuseent, ne s étend que du 4 au 6 mars... <SB> : « je vous expédie le général de paladines... <SB> et trois divisions représentant 30,000 hommes sont en marche. <SB> mais les troupes, même en chemin de fer, ne peuvent pas aller aussi vite que vous le supposez. <SB> croyez qu\'en fait de choses pareilles, rien ne sera négligé... <SB> nous allons vous renvoyer aussi picard et un ou deux de vos collèques.. <SB> il n\'est pas possible que la garde nationale n\'intervienne pas. <SB> si elle ne je fait pas, nous le ferons